<a href="https://colab.research.google.com/github/SelinZ/Biosignal-Emotions-BHS-2020/blob/master/TEZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pymatreader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
! pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 462 kB 4.1 MB/s 


In [2]:
!  pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 3.8 MB/s 


In [3]:
import os
import numpy as np
import mne
import pymatreader
%matplotlib inline
from scipy.io import loadmat 
import mne, glob 
from numpy import loadtxt
from pymatreader import read_mat
from pylab import rcParams

In [15]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [16]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

In [19]:
features = open('/Features14.csv')

# let's print what it looks like...
print(type(features))

<class '_io.TextIOWrapper'>


In [24]:
import pandas as pd
import io
 
df = pd.read_csv('/Features14.csv').drop(["Unnamed: 0"],axis=1)

print(df)

     psdtheta_1  psdalpha_1  psdbeta_1  psdtheta_2  psdalpha_2  psdbeta_2  \
0     -0.137607   -0.137238  -0.116997   -0.080612   -0.081118  -0.108773   
1     -0.139174   -0.138685  -0.117599   -0.096207   -0.096338  -0.110293   
2     -0.127690   -0.127539  -0.110420   -0.101203   -0.101259  -0.098381   
3     -0.158972   -0.158123  -0.131111   -0.100674   -0.100739  -0.097964   
4     -0.110480   -0.110713  -0.099031   -0.111506   -0.111337  -0.089051   
..          ...         ...        ...         ...         ...        ...   
409   -0.176673   -0.175428  -0.142822   -0.119956   -0.118711  -0.111964   
410   -0.143317   -0.142784  -0.120602   -0.111610   -0.111338  -0.079139   
411   -0.153161   -0.152451  -0.127275   -0.055160   -0.047706  -0.035399   
412   -0.221893   -0.219656  -0.172804   -0.092856   -0.088311  -0.070856   
413   -0.128817   -0.128628  -0.111106   -0.141103   -0.139356  -0.113828   

     psdtheta_3  psdalpha_3  psdbeta_3  psdtheta_4  ...  psdbeta_12  \
0   

In [28]:
feature_X=df.iloc[:,0:-3]
feature_V=df.iloc[:,-3]
feature_A=df.iloc[:,-2]
feature_D=df.iloc[:,-1]
feature_XVD = pd.concat([feature_X, feature_V, feature_D], axis=1)
feature_XAD = pd.concat([feature_X, feature_A, feature_D], axis=1)
feature_XAV = pd.concat([feature_X, feature_V, feature_A], axis=1)


In [25]:
sampling_rate = 128
window_size = 1280
overlap = 256
channel_len = 14
classes = 2
EEG_Electrodes = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4'] 
noOfSubjects =  23
noOfVideoSequences= 18

bands = {'delta': [0.5/(sampling_rate/2), 4/(sampling_rate/2)], 'theta': [4/(sampling_rate/2), 8/(sampling_rate/2)], \
         'alpha': [8/(sampling_rate/2), 14/(sampling_rate/2)], 'beta': [14/(sampling_rate/2), 30/(sampling_rate/2)], \
         'gamma': [30/(sampling_rate/2), 50/(sampling_rate/2)]}


In [26]:
"""We need the following information to create MNE structure:
    data ([ndarray]): [trials x chans x samples]
    y ([ndarray]):    [class label array  [1, labels]]
    sfreq ([int]):    [sampling frequency]
    event_id ([dict]): [{1 :'pos', -1 : 'neg'} - class labels id]
    chan_names ([list]): [channel names in a list of strings]
"""                     

n_channels = 14

# Initialize an info structure
info = mne.create_info(
        ch_names = EEG_Electrodes,
        ch_types = ['eeg']*n_channels,
        sfreq    = sampling_rate
        )   

info.set_montage('standard_1020')
print('Event created :', info) 

Event created : <Info | 8 non-empty values
 bads: []
 ch_names: AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, FC6, F4, F8, AF4
 chs: 14 EEG
 custom_ref_applied: False
 dig: 17 items (3 Cardinal, 14 EEG)
 highpass: 0.0 Hz
 lowpass: 64.0 Hz
 meas_date: unspecified
 nchan: 14
 projs: []
 sfreq: 128.0 Hz
>


### **CNN**

In [52]:
def create_raw_model(nchan, nclasses, trial_length=960, l1=0):
    """
    CNN model definition
    """
    input_shape = (trial_length, nchan, 1)
    model = Sequential()
    model.add(Conv2D(40, (30, 1), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="same", input_shape=input_shape))
    model.add(Conv2D(40, (1, nchan), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="valid"))
    model.add(tf.keras.layers.GlobalAveragePooling2D((30, 1)))

    model.add(Flatten())
    model.add(Dense(80, activation="relu"))
    model.add(Dense(nclasses, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
    return model

In [30]:
def fit_model(model, X, y, train_idx, test_idx, input_length=50, batch_size=32, epochs=30, steps_per_epoch=1000, callbacks=None):    
    gc.collect()
    return model.fit_generator(
        nndata.crossval_gen(X,y, train_idx, input_length, batch_size),
        validation_data=nndata.crossval_test(X, y, test_idx, input_length),
        steps_per_epoch=steps_per_epoch, epochs=epochs, callbacks=callbacks                          
    )

ERROR: unknown command "keras"


In [39]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras import regularizers

CNN 2

In [55]:
import torch
import torch.nn as nn

import numpy as np
from sklearn.model_selection import train_test_split
import argparse


In [104]:
import torch
import torch.nn as nn

class SCU(nn.Module):

    def __init__(self, opt, num_classes):

        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=9, out_channels=16, kernel_size=5, stride=2),
            nn.BatchNorm1d(num_features=16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(p=0.2))
        
        self.dense_layers = nn.Sequential(
            nn.Linear(5984, 600),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(600, 60),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(60, num_classes))

    def forward(self, x):

        out = self.layer1(x)
        out = out.view(out.size(0), -1)
        out = self.dense_layers(out)     
        return out

In [129]:
parser = argparse.ArgumentParser()
parser.add_argument('--n_epochs', type=int, default=50, help='number of epochs of training')
parser.add_argument('--lr', type=float, default=0.00001, help='adam: learning rate')
parser.add_argument('--dropout_level', type=float, default=0.5, help='dropout level')
parser.add_argument('--w_decay', type=float, default=0.001, help='weight decay')
parser.add_argument('--batch_size', type=int, default=10, help='batch size')
parser.add_argument('--seed_n', type=int, default=74, help='seed number')


_StoreAction(option_strings=['--seed_n'], dest='seed_n', nargs=None, const=None, default=74, type=<class 'int'>, choices=None, help='seed number', metavar=None)

In [132]:
from optparse import OptionParser
...
parser = OptionParser()
parser.add_option("-f", "--file", dest="filename",
                  help="write report to FILE", metavar="FILE")
parser.add_option("-q", "--quiet",
                  action="store_false", dest="verbose", default=True,
                  help="don't print status messages to stdout")

(opt, args) = parser.parse_args()

In [121]:
parser = argparse.ArgumentParser()
parser.add_argument('--n_epochs', type=float, default=50, help='number of epochs of training')
parser.add_argument('--lr', type=float, default=0.00001, help='adam: learning rate')
parser.add_argument('--dropout_level', type=float, default=0.5, help='dropout level')
parser.add_argument('--w_decay', type=float, default=0.001, help='weight decay')
parser.add_argument('--batch_size', type=float, default=10, help='batch size')
parser.add_argument('--seed_n', type=float, default=74, help='seed number')


_StoreAction(option_strings=['--seed_n'], dest='seed_n', nargs=None, const=None, default=74, type=<class 'float'>, choices=None, help='seed number', metavar=None)

In [133]:
print(type(opt))
print(opt)

<class 'optparse.Values'>
{'filename': '/root/.local/share/jupyter/runtime/kernel-1a4175cf-f310-489a-8f6d-03eb0d016136.json', 'verbose': True}


In [135]:
print(dir(opt))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_update', '_update_careful', '_update_loose', 'ensure_value', 'filename', 'read_file', 'read_module', 'verbose']


In [136]:
device  = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [137]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [138]:
torch.manual_seed(opt.seed_n)
torch.cuda.manual_seed(opt.seed_n)

AttributeError: ignored

In [139]:
num_classes = 2

In [101]:
num_classes = 2

In [140]:
def train_SCU(X_train, y_train):

    # convert NumPy Array to Torch Tensor
    train_input = torch.from_numpy(X_train)
    train_label = torch.from_numpy(y_train)

    # create the data loader for the training set
    trainset = torch.utils.data.TensorDataset(train_input, train_label)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=2, shuffle=True, num_workers=4)
    
    cnn = SCU(opt, num_classes).to(device)
    cnn.train()
    # Loss and Optimizer
    ce_loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(cnn.parameters(), lr=opt.lr, weight_decay=opt.w_decay)

    # loop through the required number of epochs
    for epoch in range(opt.n_epochs):

        # loop through the batches
        cumulative_accuracy = 0
        for i, data in enumerate(trainloader, 0):
            # format the data from the dataloader
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.float()
            
            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = cnn(inputs)

            loss = ce_loss(outputs, labels)
            loss.backward()
            optimizer.step()

            # calculate the accuracy over the training batch
            _, predicted = torch.max(outputs, 1)
            
            cumulative_accuracy += get_accuracy(labels, predicted)
    
    return cnn


In [141]:
def test_SCU(cnn, X_test, y_test):

     # convert NumPy Array to Torch Tensor
    test_input = torch.from_numpy(X_test)
    test_label = torch.from_numpy(y_test)

    # create the data loader for the test set
    testset = torch.utils.data.TensorDataset(test_input, test_label)
    testloader = torch.utils.data.DataLoader(testset, batch_size=2, shuffle=False, num_workers=4)

    cnn.eval()
    test_cumulative_accuracy = 0
    for i, data in enumerate(testloader, 0):
        # format the data from the dataloader
        test_inputs, test_labels = data
        test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
        test_inputs = test_inputs.float()    

        test_outputs = cnn(test_inputs)
        _, test_predicted = torch.max(test_outputs, 1)    
        
        test_acc = get_accuracy(test_labels,test_predicted)
        test_cumulative_accuracy += test_acc

    return test_cumulative_accuracy, len(testloader)


In [89]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 54.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f05023a2d8c02acfefdd8f0a96ccbdac065e2302251a27f2d32593d2f66e1115
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [90]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [142]:
import torch

# Just right before the actual usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cnn = SCU(opt, num_classes).to(device)
cnn.train()

SCU(
  (layer1): Sequential(
    (0): Conv1d(9, 16, kernel_size=(5,), stride=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
  )
  (dense_layers): Sequential(
    (0): Linear(in_features=5984, out_features=600, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=600, out_features=60, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=60, out_features=2, bias=True)
  )
)

In [143]:
import sklearn.model_selection 
from sklearn.model_selection import train_test_split   

if __name__ == "__main__":

    # data loading
    #Arousal->>> Valence, dominance included in the feature X

    feature_XVD = pd.concat([feature_X, feature_V, feature_D], axis=1)
    feature_A=df.iloc[:,-2]
    print(type(feature_A))
    print(type(feature_XVD))

    x5D = np.array(feature_XVD)
    y5D = np.array(feature_A)
    X_train, X_test, y_train, y_test = train_test_split(x5D, y5D, train_size=0.8)


    #feature_XVD = np.swapaxes(feature_XVD,0,1)

    # split the data  training and testing
  #  X_train, X_test, y_train, y_test = train_test_split(feature_XVD, feature_A, test_size=0.2)

    # training
    cnn = train_SCU(X_train, y_train)

    # testing
    test_cumulative_accuracy, ntest = test_SCU(cnn, X_test, y_test)

    print(f"Test Accuracy: {test_cumulative_accuracy/ntest*100}")

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


AttributeError: ignored